# YOLO Model Training and Evaluation

## Setup

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
from mss import mss

In [ ]:
base_model = "yolo11s.pt"
dataset_path="train_40/dataset.yaml"
tuned_model = "yolo11s_tuned.pt"

## Training

In [ ]:
model = YOLO(base_model)

# Train on annotated dataset
model.train(
    data=dataset_path, 
    epochs=200,
    augment=True,
    mosaic=True,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.2,
    scale=0.5,
    fliplr=0.25
)

# Save the improved model
model.save(tuned_model)

## Live Labeling

In [ ]:
def process_frame(model, frame):
    img = cv2.cvtColor(np.array(frame), cv2.COLOR_BGRA2BGR)
    
    # Run inference
    results = model.predict(img, conf=0.5)

    # Draw bounding boxes
    for box in results[0].boxes:
        x_min, y_min, x_max, y_max = map(int, box.xyxy[0].tolist())
        class_id = int(box.cls[0])
        confidence = float(box.conf[0])

        # Draw rectangle
        cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

        # Draw label
        label = f"{model.names[class_id]} {confidence:.2f}"
        cv2.putText(img, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return img

def live_labeling():
    model = YOLO(tuned_model)

    # Screen capture configuration
    monitor = {"top": 0, "left": 0, "width": 1920, "height": 1080}
    sct = mss()

    cv2.namedWindow("Live Labels", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Live Labels", 960, 540)

    while True:
        # Capture screen
        screen = sct.grab(monitor)
        
        # Process frame
        labeled_frame = process_frame(model, screen)

        # Display labeled frame
        cv2.imshow("Live Labels", labeled_frame)

        # Exit
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    # Cleanup
    cv2.destroyAllWindows()

live_labeling()